# EnergyPlus Parameterization and Sensitivity Analysis for Residential Single Family Homes


## Daniel Xu, Keller Lab

This notebook was created in order to document the workflow and processes of EnergyPlus within Python. This is for my project involving the parameterization of EnergyPlus. Given that EnergyPlus has over 2000 available inputs, more research needs to be done in order to document which parameters are most significant and relevant in home energy modeling and simulations. 

The goal of this project is to run EnergyPlus on a model single family residential home. Some of the inputs will be fixed to default variables. Other variables, such as window thickness, thermostat set-points, etc. will be varied on pre-defined distributions. 

The distributions will be defined as seen in this paper: 
https://www.sciencedirect.com/science/article/abs/pii/S037877881631372X?via=ihub

### Importing necessary libraries

In [1]:
from eppy import modeleditor
from eppy.modeleditor import IDF
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

### Changing the working directory 

In [2]:
os.chdir("/Users/danielxu/Desktop/Dartmouth College/5. Keller Lab/24S/eplus_sensitivity/scripts")
print("Current working directory:", os.getcwd())

Current working directory: /Users/danielxu/Desktop/Dartmouth College/5. Keller Lab/24S/eplus_sensitivity/scripts


### Setting EnergyPlus Input Data Dictionary

In [3]:
iddfile = "/Applications/EnergyPlus-24-1-0/Energy+.idd"
IDF.setiddname(iddfile)

### Setting a Skeleton IDF File 

This modifiable IDF file is provided by EnergyPlus in its initial download. It provides a mock residential home in Chicago with multiple zones. I will assume that some of the inputs are fixed and others are varied on a distribution. 

Directly from the EnergyPlus documentation: 

"This file does the basic test of an air distribution system in a residential home. A two speed heat pump with a supplmental gas heater provides space heating and cooling. It provides ventilation through the ZoneAirBalance:OutdoorAir model."

In [4]:
idfname = "/Applications/EnergyPlus-24-1-0/ExampleFiles/SingleFamilyHouse_TwoSpeed_CutoutTemperature.idf"

preidf = IDF(idfname)

# preidf.printidf()

print(preidf.idfobjects['Site:Location'])

[
Site:Location,
    CHICAGO_IL_USA TMY2-94846,    !- Name
    41.78,                    !- Latitude
    -87.75,                   !- Longitude
    -6,                       !- Time Zone
    190;                      !- Elevation
]


## Creating a hot water system in the IDF file

In [13]:
water_heater = preidf.newidfobject("WATERHEATER:MIXED")
water_heater.Name = "New Water Heater"
water_heater.Tank_Volume = "0.15"  # cubic meters
water_heater.Heater_Maximum_Capacity = "2000"  # watts
water_heater.Heater_Minimum_Capacity = "1000"  # watts
water_heater.Setpoint_Temperature_Schedule_Name = "Hot Water Setpoint Temp"
water_heater.Use_Side_Inlet_Node_Name = "Hot Water Use Inlet Node"
water_heater.Use_Side_Outlet_Node_Name = "Hot Water Use Outlet Node"
water_heater.Source_Side_Inlet_Node_Name = "Hot Water Source Inlet Node"
water_heater.Source_Side_Outlet_Node_Name = "Hot Water Source Outlet Node"
water_heater.Ambient_Temperature_Indicator = "Zone"  # Modify as needed: Options are 'Zone', 'Schedule', or 'OutdoorAirNode'
water_heater.Heater_Fuel_Type = "Electricity"  # Modify as needed: common types include 'Electricity', 'NaturalGas', etc.
water_heater.Heater_Thermal_Efficiency = "0.92"  # Fractional efficiency

# Save the IDF file
preidf.save("/Users/danielxu/Desktop/Dartmouth College/5. Keller Lab/24S/eplus_sensitivity/scripts/skeleton.idf")

### Weather Station Data Retrieval for EnergyPlus Simulations

This project aims to enhance the accuracy of building energy simulations by sourcing precise local weather data. By leveraging the NCEI (National Centers for Environmental Information) API, we are able to identify and retrieve data from the nearest weather station to any specified location. This localized weather data is crucial for feeding into EnergyPlus simulations, ensuring that our building models operate under realistic environmental conditions. The accurate simulation of energy usage and needs based on actual weather data helps in designing more efficient and sustainable buildings.

For more detailed information on how to utilize the NCEI API, including obtaining access tokens and making API requests, please visit the NCEI Web Services Documentation.

In [6]:
import requests
from datetime import datetime, timedelta

def get_active_weather_stations(api_key, lat, lon):
    base_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/stations'
    headers = {'token': api_key}
    # Define the date one year ago from today
    one_year_ago = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

    # Set the request to retrieve more stations for a broader check
    params = {
        'extent': f'{lat-0.05},{lon-0.05},{lat+0.05},{lon+0.05}',
        'limit': '1000',  # Adjust limit as needed
        'sortfield': 'mindate',
        'sortorder': 'desc'
    }

    response = requests.get(base_url, headers=headers, params=params)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: {response.status_code} - {response.text}")
        return []

    try:
        data = response.json()
        active_stations = []
        # Filter stations by checking if the 'maxdate' is within the last year
        for station in data.get('results', []):
            if station['maxdate'] >= one_year_ago:
                active_stations.append(station)
        return active_stations
    except ValueError:
        print("Failed to decode JSON from response.")
        return []

api_key = 'SrgpVmvZhbtZXRSdBgknhaRSQlhTNzBt'
latitude = 41.78  
longitude = -87.75  

active_stations = get_active_weather_stations(api_key, latitude, longitude)
print(f"Found {len(active_stations)} active stations:")
for station in active_stations:
    print(station)


Found 4 active stations:
{'elevation': 191.1, 'mindate': '2009-04-01', 'maxdate': '2024-05-15', 'latitude': 41.730399, 'name': 'OAK LAWN 1.1 N, IL US', 'datacoverage': 0.9999, 'id': 'GHCND:US1ILCK0139', 'elevationUnit': 'METERS', 'longitude': -87.75615}
{'elevation': 185.8, 'mindate': '1997-05-01', 'maxdate': '2024-05-19', 'latitude': 41.78412, 'name': 'CHICAGO MIDWAY AIRPORT, IL US', 'datacoverage': 1, 'id': 'GHCND:USW00014819', 'elevationUnit': 'METERS', 'longitude': -87.75514}
{'elevation': 185.8, 'mindate': '1948-01-01', 'maxdate': '2024-05-22', 'latitude': 41.78412, 'name': 'CHICAGO MIDWAY AIRPORT, IL US', 'datacoverage': 1, 'id': 'WBAN:14819', 'elevationUnit': 'METERS', 'longitude': -87.75514}
{'elevation': 189, 'mindate': '1928-02-01', 'maxdate': '2024-05-20', 'latitude': 41.73727, 'name': 'CHICAGO MIDWAY AIRPORT 3 SW, IL US', 'datacoverage': 0.9993, 'id': 'GHCND:USC00111577', 'elevationUnit': 'METERS', 'longitude': -87.77734}


Based on this location, we will then use the diyepw package to create time series weather data. More information about diyepw can be found at: 

https://diyepw.readthedocs.io/en/latest/tutorial.html (PNNL)

In [7]:
import diyepw

diyepw.create_amy_epw_files_for_years_and_wmos(
  [2020,2021,2022,2023],
  [725340],
  max_records_to_interpolate=6,
  max_records_to_impute=48,
  max_missing_amy_rows=5,
  allow_downloads=True,
  amy_epw_dir='./'
)

2024-05-23 15:45:39,651 Creating AMY EPW for year 2020 and WMO 725340...
2024-05-23 15:45:39,651 No amy_dir was specified - downloaded AMY files will be stored in the default location at /Users/danielxu/ENTER/envs/energyplus/lib/python3.9/site-packages/diyepw/data/noaa_isd_lite_files
2024-05-23 15:45:39,685 TMY EPW file (/Users/danielxu/ENTER/envs/energyplus/lib/python3.9/site-packages/diyepw/data/tmy_epw_files/USA_IL_Chicago.Midway.Intl.AP.725340_TMY3.epw) already exists, won't download again.
2024-05-23 15:45:39,700 2020 is a leap year, using the interpolation strategy to populate TMY data for Feb. 29
2024-05-23 15:45:39,749 File already exists at ./USA_IL_Chicago-Midway-Intl-AP.725340_AMY_2020.epw, so a new one won't be generated.
2024-05-23 15:45:39,750 Creating AMY EPW for year 2021 and WMO 725340...
2024-05-23 15:45:39,750 No amy_dir was specified - downloaded AMY files will be stored in the default location at /Users/danielxu/ENTER/envs/energyplus/lib/python3.9/site-packages/diy

{2020: {725340: ['./USA_IL_Chicago-Midway-Intl-AP.725340_AMY_2020.epw']},
 2021: {725340: ['./USA_IL_Chicago-Midway-Intl-AP.725340_AMY_2021.epw']},
 2022: {725340: ['./USA_IL_Chicago-Midway-Intl-AP.725340_AMY_2022.epw']},
 2023: {725340: ['./USA_IL_Chicago-Midway-Intl-AP.725340_AMY_2023.epw']}}

### Editing IDF parameters 



In [9]:
import re

def update_building_parameters(idf_file_path, idd_file_path, output_idf_path):
    # Load the EnergyPlus IDD file
    IDF.setiddname(idd_file_path)

    # Load the IDF file
    idf = IDF(idf_file_path)

    # Retrieve the specific PEOPLE object
    people_object = idf.getobject('PEOPLE', 'LIVING ZONE People')
    # Generate a new value for People per Zone Floor Area using numpy's uniform distribution
    new_people_per_area = np.random.uniform(0.002, 0.060)  # People per m2
    # Update the People object
    people_object.People_per_Floor_Area = new_people_per_area
    # Print the updated settings for verification
    print(f"Updated {people_object.Name} with People per Zone Floor Area = {new_people_per_area} pp/m2")

    # Retrieve the specific ZoneInfiltration:DesignFlowRate object
    infiltration_object = idf.getobject('ZONEINFILTRATION:DESIGNFLOWRATE', 'LIVING ZONE Infil 1')
    # Generate a new value for Flow per Zone Floor Area using numpy's uniform distribution
    new_flow_per_area = np.random.uniform(0.0, 1.5)  # m3/s/m2
    # Update the infiltration object
    infiltration_object.Design_Flow_Rate_Calculation_Method = 'Flow/Area'
    infiltration_object.Design_Flow_Rate = new_flow_per_area
    # Print the updated settings for verification
    print(f"Updated {infiltration_object.Name} with Flow per Zone Floor Area = {new_flow_per_area} m3/s-m2")

    # Update Equipment Power Density
    equipment_object = idf.getobject('ELECTRICEQUIPMENT', 'LIVING ZONE ElecEq')
    new_design_level = np.random.uniform(1, 40)  # Watts
    equipment_object.Design_Level = new_design_level
    print(f"Updated {equipment_object.Name} with Design Level = {new_design_level} W")

    # Retrieve and update the WATERHEATER:MIXED object
    water_heater = idf.getobject('WATERHEATER:MIXED', 'New Water Heater')
    # Generate a new value for Domestic Hot Water Flow Rate using numpy's uniform distribution
    new_dhw_flow_rate = np.random.uniform(1e-8, 20e-8)  # m3/s
    water_heater.Use_Side_Design_Flow_Rate = new_dhw_flow_rate
    print(f"Updated {water_heater.Name} with Domestic Hot Water Flow Rate = {new_dhw_flow_rate} m3/s")

  # Generate new temperature setpoints
    new_heating_setpoint = np.random.uniform(15.0, 25.0)  # Uniform distribution for heating setpoint
    new_cooling_setpoint = np.random.uniform(23.0, 29.0)  # Uniform distribution for cooling setpoint

    # Retrieve and update the heating setpoint schedule
    heating_schedule = idf.getobject('SCHEDULE:COMPACT', 'Dual Heating Setpoints')
    # Assuming Field 4 contains the setpoint, following your latest IDF file structure
    heating_schedule.fieldvalues[6] = str(new_heating_setpoint)  # Index 5 corresponds to Field 4
    print(f"Updated Heating Setpoint to {new_heating_setpoint}°C")

    # Retrieve and update the cooling setpoint schedule
    cooling_schedule = idf.getobject('SCHEDULE:COMPACT', 'Dual Cooling Setpoints')
    cooling_schedule.fieldvalues[6] = str(new_cooling_setpoint)  # Index 5 corresponds to Field 4
    print(f"Updated Cooling Setpoint to {new_cooling_setpoint}°C")

    # Save the modified IDF file
    idf.save(output_idf_path)

# Paths to your IDF and IDD files
idf_file_path = "/Users/danielxu/Desktop/Dartmouth College/5. Keller Lab/24S/eplus_sensitivity/scripts/skeleton.idf"
#idf_file_path = "/Applications/EnergyPlus-24-1-0/ExampleFiles/SingleFamilyHouse_TwoSpeed_MultiStageElectricSuppCoil.idf"
idd_file_path = "/Applications/EnergyPlus-24-1-0/Energy+.idd"
output_idf_path = "/Users/danielxu/Desktop/Dartmouth College/5. Keller Lab/24S/eplus_sensitivity/scripts/randomized.idf"

# Execute the function
update_building_parameters(idf_file_path, idd_file_path, output_idf_path)

Updated LIVING ZONE People with People per Zone Floor Area = 0.04458981029574084 pp/m2
Updated LIVING ZONE Infil 1 with Flow per Zone Floor Area = 0.42705004354678944 m3/s-m2
Updated LIVING ZONE ElecEq with Design Level = 6.053617158975488 W
Updated New Water Heater with Domestic Hot Water Flow Rate = 1.924551067891771e-07 m3/s
Updated Heating Setpoint to 20.592523065031237°C
Updated Cooling Setpoint to 25.983840341687895°C


In [12]:
runidf= IDF(output_idf_path, "/Applications/EnergyPlus-24-1-0/WeatherData/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw")
runidf.run()


/Applications/EnergyPlus-24-1-0/energyplus --weather /Applications/EnergyPlus-24-1-0/WeatherData/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw --output-directory /Users/danielxu/Desktop/Dartmouth College/5. Keller Lab/24S/eplus_sensitivity/scripts --idd /Applications/EnergyPlus-24-1-0/Energy+.idd /Users/danielxu/Desktop/Dartmouth College/5. Keller Lab/24S/eplus_sensitivity/scripts/randomized_80dc64.idf

EnergyPlus Starting
EnergyPlus, Version 24.1.0-9d7789a3ac, YMD=2024.05.23 17:18
**FATAL:Errors occurred on processing input file. Preceding condition(s) cause termination.
EnergyPlus Run Time=00hr 00min  0.03sec


Program terminated: EnergyPlus Terminated--Error(s) Detected.


EnergyPlusRunError: 

Contents of EnergyPlus error file at /Users/danielxu/Desktop/Dartmouth College/5. Keller Lab/24S/eplus_sensitivity/scripts/eplusout.err
Program Version,EnergyPlus, Version 24.1.0-9d7789a3ac, YMD=2024.05.23 17:18,
   ** Severe  ** <root>[WaterHeater:Mixed][New Water Heater] - Missing required property 'ambient_temperature_indicator'.
   ** Severe  ** <root>[WaterHeater:Mixed][New Water Heater] - Missing required property 'heater_fuel_type'.
   ** Severe  ** <root>[WaterHeater:Mixed][New Water Heater] - Missing required property 'heater_thermal_efficiency'.
   **  Fatal  ** Errors occurred on processing input file. Preceding condition(s) cause termination.
   ...Summary of Errors that led to program termination:
   ..... Reference severe error count=3
   ..... Last severe error=<root>[WaterHeater:Mixed][New Water Heater] - Missing required property 'heater_thermal_efficiency'.
   ************* Warning:  Node connection errors not checked - most system input has n